The baseline code does not work if no test labels are provided, so we assign random ones in order to reproduce it.

In [48]:
import pandas as pd

In [49]:
DATA_FOLDER = "/mnt/thau03a/datasets/c1_muse_perception"

In [50]:
labels = pd.read_csv(f"{DATA_FOLDER}/pca_labels.csv")
print(labels.head())
print(labels.shape)
print(labels.describe())

   subj_id  aggressive  arrogant  attractive  charismatic  competitive  \
0        2      0.4940    0.4524      0.5595       0.6667       0.6905   
1        3      0.3944    0.3000      0.5000       0.4611       0.5778   
2        9      0.3529    0.3382      0.6275       0.5098       0.6324   
3       10      0.4943    0.4195      0.5632       0.5517       0.6667   
4       13      0.3333    0.2944      0.6667       0.6000       0.6167   

   dominant  enthusiastic  expressive  friendly  ...  confident  independent  \
0    0.6429        0.6964      0.8333    0.5774  ...     0.8155       0.7976   
1    0.4611        0.4667      0.4444    0.5778  ...     0.7222       0.6778   
2    0.4902        0.4804      0.5735    0.5539  ...     0.7451       0.6520   
3    0.4828        0.5402      0.6264    0.6034  ...     0.7529       0.7414   
4    0.5667        0.5278      0.5944    0.7000  ...     0.7722       0.7167   

     risk  sincere  collaborative    kind    warm  good_natured       PC1 

In [51]:
PARTITION_PATH = f"{DATA_FOLDER}/metadata/partition.csv"
partition = pd.read_csv(PARTITION_PATH)
print(partition.head())
print(partition.shape)
print(partition.describe())

   Id Partition
0   0      test
1   1      test
2   2     train
3   3     train
4   6      test
(177, 2)
               Id
count  177.000000
mean   121.920904
std     68.302922
min      0.000000
25%     62.000000
50%    126.000000
75%    182.000000
max    231.000000


In [52]:
# Add a NaN row for each missing subject_ids in labels
partition_subject_ids = partition["Id"].unique()
labels_subject_ids = labels["subj_id"].unique()

missing_subject_ids = set(partition_subject_ids) - set(labels_subject_ids)
print(f"Missing subject_ids: {[str(x) for x in missing_subject_ids]}")
missing_rows = []
for subject_id in missing_subject_ids:
    missing_rows.append({"subj_id": subject_id})
missing_df = pd.DataFrame(missing_rows)
labels = pd.concat([labels, missing_df], ignore_index=True)
labels.tail()

Missing subject_ids: ['0', '1', '130', '6', '8', '136', '140', '16', '144', '153', '26', '154', '155', '33', '165', '38', '40', '44', '46', '49', '177', '181', '54', '183', '184', '185', '63', '64', '191', '66', '192', '193', '69', '70', '194', '195', '196', '198', '75', '199', '77', '203', '80', '81', '208', '83', '210', '214', '220', '221', '101', '229', '231', '108', '109', '112', '116', '117', '124', '126']


,subj_id,aggressive,arrogant,attractive,charismatic,competitive,dominant,enthusiastic,expressive,friendly,...,confident,independent,risk,sincere,collaborative,kind,warm,good_natured,PC1,PC2
172,112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Are the subj_id in labels the same as the Ids in partition?
labels_subject_ids = labels["subj_id"].unique()
partition_subject_ids = partition["Id"].unique()

print(f"Number of unique subj_id in labels: {len(labels_subject_ids)}")
print(f"Number of unique Id in partition: {len(partition_subject_ids)}")

print(f"Are the subj_id in labels the same as the Ids in partition? {set(labels_subject_ids) == set(partition_subject_ids)}")

Number of unique subj_id in labels: 177
Number of unique Id in partition: 177
Are the subj_id in labels the same as the Ids in partition? True


In [54]:
# are the nan rows in labels just the subj_ids that are in the partition test set?
partition_test_subject_ids = partition[partition["Partition"] == "test"]["Id"].unique()
# Get the rows with NaN values
nan_rows = labels[labels.isnull().any(axis=1)]
nan_rows_subject_ids = nan_rows["subj_id"].unique()

print(f"Number of unique subj_id in nan_rows: {len(nan_rows_subject_ids)}")
print(f"Number of unique Id in partition test set: {len(partition_test_subject_ids)}")

print(f"Are the subj_id in nan_rows the same as the Ids in partition test set? {set(nan_rows_subject_ids) == set(partition_test_subject_ids)}")

Number of unique subj_id in nan_rows: 60
Number of unique Id in partition test set: 60
Are the subj_id in nan_rows the same as the Ids in partition test set? True


In [55]:
OUT_PATH = f"{DATA_FOLDER}/pca_labels_with_test.csv"
labels.to_csv(OUT_PATH, index=False)